# Installing Required Packages

In [1]:
!pip install -U autoawq

  Attempting uninstall: nvidia-cusolver-cu12
    Found existing installation: nvidia-cusolver-cu12 11.6.3.83
    Uninstalling nvidia-cusolver-cu12-11.6.3.83:
      Successfully uninstalled nvidia-cusolver-cu12-11.6.3.83
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


# Defining Prompts

In [1]:
system_prompt = """
You are a JSON-only response system. Follow these rules absolutely:
1. ONLY output valid, parseable JSON
2. NEVER include text before or after the JSON
3. NEVER include markdown code blocks or formatting
4. NEVER include explanations
5. NEVER extract dates
6. If you can't fulfill a request, return {"error": "error message"}
7. Output should always be a single JSON object

For address requests, use this format:
{
    "address": {
        "license": "B1231241",
        "Address": "X City",
        "Sex": "Male",
        "Weight": "X",
        "Height": "X"
    }
}
"""

In [2]:
test_instruction = '''extract NER:
        California
        DRIVER LICENSe
        dl 11234568
        CLASS C
        EXP 08/31/2014
        END NONE
        LNCARDHOLDER FNIMA
        2570 24TH STREET ANYTOWN, CA 95818
        doB 08/31/1977 RSTR NONE
        08311977
        VETERAN
        Cordhslde
        SEX F HGT 5'-05"
        HAIR BRN WGT 125 lb
        EYES BRN
        DD 00/00/0000NNNAN/ANFD/YY
        ISS 08/31/2009
'''

AWQ

In [3]:
import torch
import time
from transformers import AutoModelForCausalLM, AutoTokenizer

# Check if CUDA is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load model and tokenizer
model_name = "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct-AWQ"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16 if device == "cuda" else torch.float32,
    trust_remote_code=True,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define your prompts
# system_prompt = "You are a helpful AI assistant."
# test_instruction = "Explain what machine learning is in simple terms."

# Prepare messages
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": test_instruction}
]

# Tokenize input
input_ids = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
)

# Generate response
start_time = time.time()
with torch.no_grad():  # Save memory during inference
    output = model.generate(
        input_ids.to(device),
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=128,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id  # Handle padding
    )
end_time = time.time()
execution_time = end_time - start_time

# Decode and print only the new tokens
response = tokenizer.decode(output[0][input_ids.shape[1]:], skip_special_tokens=True)
print("Response:", response)
print(f"Execution time: {execution_time:.2f} seconds")

Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch.bfloat16` is not supported for AWQ kernels yet. Casting to `torch.float16`.
/usr/local/lib/python3.11/dist-packages/awq/__init__.py:21: DeprecationWarning: 
I have left this message as the final dev message to help you transition.

Important Notice:
- AutoAWQ is officially deprecated and will no longer be maintained.
- The last tested configuration used Torch 2.6.0 and Transformers 4.51.3.
- If future versions of Transformers break AutoAWQ compatibility, please report th

Response: ```json
{
    "NER": {
        "Location": {
            "State": "California",
            "Street": "24TH STREET",
            "City": "ANYTOWN",
            "Zip": "95818"
        },
        "Identification": {
            "License": "DL 11234568",
            "Class": "C",
            "Expiration": "08/31/2014",
            "LicenseHolder": {
                "Name": "FNIMA",

Execution time: 11.88 seconds
